# Homework 1: Filtering spam messages

**Ing 5 BDA grp1:**

Antoniadis Pablo

LLobregat Thomas

Ye Sebastien


# 0. Import the dataset from message.txt


*   import numpy as np
*   import pandas as pd
*   import matplotlib.pyplot as plt


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

path = "/content/drive/My Drive/Ing5/Machine learning/homeworks/HM 1: Filtering spam messages/"

os.chdir(path)

with open(path+'messages.txt', 'r') as db:
  data = db.readlines()
 

Mounted at /content/drive


In [2]:
 # little bit of data cleaning

 #remove '\n' at the end of each email  
data_length = len(data)
for i in range(data_length):
  data[i] = data[i][:-2]

# 1. Divide the data in two groups: training and test examples.

We decided to divide the data in two groups the training_set that has 70% of the data and test_set that has 30% 

In [3]:
data_length = len(data) #5000

training_set = [] # 70% = 3500
test_set = [] #30% = 1500

for i in range(data_length):
  if(i<data_length*0.7):
    training_set.append(data[i])
  else:
    test_set.append(data[i])
  
print("training_set length : ",len(training_set),"\ntest_set length : ",len(test_set))

training_set length :  3500 
test_set length :  1500


# 2. Parse both the training and test examples to generate both the spam and ham data sets.

In [4]:
X_train = []
y_train = []



for i in range(len(training_set)):
  y , X = training_set[i].split('\t')
  y_train.append(y)
  X_train.append(X)




In [5]:
X_test = []
y_test = []

for i in range(len(test_set)):
  y , X = test_set[i].split('\t')
  y_test.append(y)
  X_test.append(X)

# 3. Generate a dictionary from the training data.


In [6]:
training_data_frame = pd.DataFrame(list(zip(y_train, X_train)), 
               columns =['value => y_train', 'mail => X_train']) 

training_data_frame.head()

,value => y_train,mail => X_train
0,ham,Yup i've finished c ü there..
1,ham,Remember to ask alex about his pizz
2,ham,No da..today also i forgot.
3,ham,Ola would get back to you maybe not today but ...
4,ham,Fwiw the reason I'm only around when it's time...


In [7]:
test_data_frame = pd.DataFrame(list(zip(y_test, X_test)), 
               columns =['value => y_test', 'mail => X_test']) 


test_data_frame.head()



,value => y_test,mail => X_test
0,ham,Ok. She'll be ok. I gues
1,ham,aathi..where are you dear.
2,ham,Any pain on urination any thing else
3,ham,7 at esplanade.. Do ü mind giving me a lift co...
4,ham,I wnt to buy a BMW car urgently..its vry urgen...


# 4. Extract features from both the training data and test data.

## Getting all unique words first

In [8]:
words_set = set()

for mail in training_data_frame["mail => X_train"]:
  words = mail.split()
  for word in words:
    words_set.add(word)

In [9]:
training_data_frame = pd.concat(
    [
        training_data_frame,
        pd.DataFrame(
            columns=list(words_set)
        )
    ], axis=1
)

In [10]:
test_data_frame = pd.concat(
    [
        test_data_frame,
        pd.DataFrame(
            columns=list(words_set)
        )
    ], axis=1
)

In [11]:
training_data_frame.fillna(0, inplace=True)
test_data_frame.fillna(0, inplace=True)

In [12]:
for index, row in training_data_frame.iterrows():
  words = row['mail => X_train'].split()
  for word in words:
    training_data_frame.at[index, word] += 1

In [13]:
for index, row in test_data_frame.iterrows():
  words = row['mail => X_test'].split()
  for word in words:
    if word in test_data_frame.columns:
      test_data_frame.at[index, word] += 1

In [14]:
training_data_frame.head(5)

,value => y_train,mail => X_train,upset,hair..,him:,this,SLAP,tihs,21870000>Hi,abj,babes,awkward.,Indians,demand,love.rakhesh,EVEN,Police.,Things,num,Message:some,sandiago,dismissial,sariyag,"Cost£1.50/pm,",everything,HONEY?DID,thesis,FIND,Before,Hrishi,meals,ese,"Big,",propsd,smile..:),W4,&gt;:,fishrman,Ten,SURPRISE,...,apology,frnd,PRIVATE!,Direct,Sirji.I,Oz?,Help0871872887,The,are,that),morning:,accomodate,Waste,smile*,DOESDISCOUNT!SHITINNIT,DEAD,nam,blah...,meets,crea,TONITE,reply...,SPINOUT!,cannot,horniest,Hi,SONY,required:,show.,Wrong,MobStoreQuiz10pp,di,kadeem,today...!,uses,aids,uploaded,bars,Call2OptOut/67,vibrant
0,ham,Yup i've finished c ü there..,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,ham,Remember to ask alex about his pizz,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,ham,No da..today also i forgot.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,ham,Ola would get back to you maybe not today but ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,ham,Fwiw the reason I'm only around when it's time...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
test_data_frame.head(5)

,value => y_test,mail => X_test,upset,hair..,him:,this,SLAP,tihs,21870000>Hi,abj,babes,awkward.,Indians,demand,love.rakhesh,EVEN,Police.,Things,num,Message:some,sandiago,dismissial,sariyag,"Cost£1.50/pm,",everything,HONEY?DID,thesis,FIND,Before,Hrishi,meals,ese,"Big,",propsd,smile..:),W4,&gt;:,fishrman,Ten,SURPRISE,...,apology,frnd,PRIVATE!,Direct,Sirji.I,Oz?,Help0871872887,The,are,that),morning:,accomodate,Waste,smile*,DOESDISCOUNT!SHITINNIT,DEAD,nam,blah...,meets,crea,TONITE,reply...,SPINOUT!,cannot,horniest,Hi,SONY,required:,show.,Wrong,MobStoreQuiz10pp,di,kadeem,today...!,uses,aids,uploaded,bars,Call2OptOut/67,vibrant
0,ham,Ok. She'll be ok. I gues,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,ham,aathi..where are you dear.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,ham,Any pain on urination any thing else,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,ham,7 at esplanade.. Do ü mind giving me a lift co...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,ham,I wnt to buy a BMW car urgently..its vry urgen...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
training_data_frame.drop(columns=["mail => X_train"], inplace=True)
training_data_frame['value => y_train'].replace(["ham", "spam"], [0, 1], inplace=True) #replacing ham by 0 and spam by 1

In [17]:
test_data_frame.drop(columns=["mail => X_test"], inplace=True)
test_data_frame['value => y_test'].replace(["ham", "spam"], [0, 1], inplace=True) #replacing ham by 0 and spam by 1

# 5. Implement the Naive Bayes from scratch, fit the respective models to the training data.

In [18]:
class NaiveBayes():
  def __init__(self, y):
    self.num_classes = len(np.unique(y)) # Get number of classes (number of unique value in y) = 2 (spam=1 and ham=0)
    self.alpha = 1

  def fit(self, X, y):
    c = y.value_counts() # Counting the total number of each class
    self.total_ham = c[0]
    self.total_spam = c[1]
    self.ham_prop = self.total_ham / (self.total_ham + self.total_spam)
    self.spam_prop = self.total_spam / (self.total_ham + self.total_spam)
    self.word_count_per_class = pd.concat([X, y], axis=1).groupby([y.name]).sum() # Creating a dataframe in which the total number of each word per class is stored

    for count in self.word_count_per_class.iloc[0]:
      (self.alpha + count )/(self.total_ham+self.alpha*self.total_ham)

    for count in self.word_count_per_class.iloc[1]:
      (self.alpha + count )/(self.total_spam+self.alpha*self.total_spam)

  def predict(self, X):
    probs = np.zeros((X.shape[0], 2)) # Array storing the results

    for index, row in X.iterrows(): # Looping through each mail
      x_ham = self.ham_prop
      x_spam = self.spam_prop

      for word in row[row != 0].iteritems(): # Looping through each word of a mail (only word that have a count that is not null)
        x_spam *= self.word_count_per_class[word[0]][1] if self.word_count_per_class[word[0]][1] != 0 else 1
        x_ham *= self.word_count_per_class[word[0]][0] if self.word_count_per_class[word[0]][0] != 0 else 1

      # Setting the scores for the mail at index
      probs[index][0] = x_ham / (x_ham + x_spam)
      probs[index][1] = x_spam / (x_spam + x_ham)

    # returning only the highest probability for each mail
    return np.argmax(probs, 1)





In [19]:
NB = NaiveBayes(training_data_frame["value => y_train"])

In [20]:
NB.fit(training_data_frame.drop(columns=["value => y_train"]), training_data_frame["value => y_train"])

In [21]:
y_pred = NB.predict(training_data_frame.drop(columns=["value => y_train"]))

In [22]:
sum(y_pred == training_data_frame["value => y_train"])/len(y_pred)


0.9502857142857143

# 6. Make predictions for the test data.

In [23]:
y_pred = NB.predict(test_data_frame.drop(columns=["value => y_test"]))

In [24]:
sum(y_pred == test_data_frame["value => y_test"])/len(y_pred)


0.9333333333333333

# 7. Measure the spam-filtering performance for each approach through the confusion matrix.

In [25]:
y_true = test_data_frame["value => y_test"]

# Like we are searching for spams we do consider spam as positive
true_ham = 0 # true negatives
false_ham = 0 # false negatives
true_spam = 0 # true positive
false_spam = 0 # false positive

for i in range(len(y_true)):
  if y_true[i] == y_pred[i]:
    if y_true[i] == 0:
      true_ham += 1
    else:
      true_spam += 1
  else:
    if y_pred[i] == 0:
      false_ham += 1
    else:
      false_spam += 1

print("true spam : ",true_spam, " |   false spam : ",false_spam )
print("----------------------------------------")
print("false ham : ",false_ham, "  |   True ham   : ",true_ham )

true spam :  106  |   false spam :  0
----------------------------------------
false ham :  100   |   True ham   :  1294


# 8. Discuss your results.

In view of our results, we can conclude that Naive Bayes Algorithm has a fairly good accuracy in training and testing, but we observe that it has a slight tendency to not filter spam enough and let some of it pass.

In order to improve our results, it would be conceivable to clean more effectively the data which currently includes noise due to punctuation and breakage in particular.
It would also be possible to extend the dataset.